# MaxCut

The cost function for the MaxCut problem as defined in the [original QAOA paper](https://arxiv.org/abs/1411.4028) is

$$
    \hat C = \frac 12 \sum_{(i, j) \in E(G)} (1 - \hat Z_i \hat Z_j),
$$

where $E(G)$ is the set of edges of the graph $G$.

In [ ]:
using QAOA, LinearAlgebra
import Random, Distributions

using PyPlot
# PyPlot.plt.style.use("paper.mplstyle")
using PyCall
nx = pyimport("networkx");

__Defining the problem by hand:__

In [ ]:
N = 4
graph = nx.cycle_graph(N) 

figure(figsize=(3, 2))
nx.draw(graph, with_labels=true)

Note that we have to __shift the edges by 1__ when going from Python to Julia:

In [ ]:
h = zeros(N)
J = zeros(N, N)
for edge in graph.edges
    J[(edge .+ (1, 1))...] = -1/2.
end
J

In [ ]:
p = 1
max_cut_problem = QAOA.Problem(p, h, J)

__Using the wrapper function:__

In [ ]:
max_cut_problem = QAOA.max_cut(N, [edge .+ (1, 1) for edge in graph.edges], num_layers=p)

__Gradient optimization with [Zygote](https://fluxml.ai/Zygote.jl/latest/):__

In [ ]:
learning_rate = 0.01
cost, params, probs = QAOA.optimize_parameters(max_cut_problem, vcat([0.5 for _ in 1:p], [0.5 for _ in 1:p]); learning_rate=learning_rate)

__Optimization with [NLopt](https://nlopt.readthedocs.io/en/latest/):__

In [ ]:
cost, params, probs = QAOA.optimize_parameters(max_cut_problem, vcat([0.5 for _ in 1:p], [0.5 for _ in 1:p]), :LN_COBYLA)

In [ ]:
xlabels = []
for bstr in digits.(0:2^N-1, base=2, pad=N)
    push!(xlabels, "\$|" * prod([string(b) for b in bstr]) * "\\rangle\$")
end

figure(figsize=(5, 3.2))
ax = subplot(111)
bar(0:2^N-1, probs)
ax.set_xticks(0:2^N-1)
ax.set_xticklabels(xlabels, rotation=90)
minorticks_off()
tight_layout()

The states `5` $\equiv |1010\rangle$ and `10` $\equiv |0101\rangle$ are indeed the correct solutions!

## Mean-Field Approximation

In [ ]:
# schedule
p = 100
τ = 0.5
γ = τ .* ((1:p) .- 1/2) ./ p |> collect
β = τ .* (1 .- (1:p) ./ p) |> collect
β[p] = τ / (4 * p)

times = range(0, 1, p+1);

In [ ]:
mf_problem = Problem(p, J)

In [ ]:
# initial spins
S = [[[1., 0., 0.] for _ in 1:N-1] for _ in 1:p+1]

# evolution with history
S = evolve!(S, mf_problem.local_fields, mf_problem.couplings, β, γ);

In [ ]:
# helper function to reformat the data
get_spin_data = n -> mapreduce(permutedims, vcat, [S[k][n] for k in 1:p+1]) |> transpose;

In [ ]:
# helper function to reformat the data
get_spin_data = n -> mapreduce(permutedims, vcat, [S[k][n] for k in 1:p+1]) |> transpose;

In [ ]:
# plot x, y, and z of all spins 
figure(figsize=((N - 1) * 2.2, 2))

for n in 1:N - 1
    subplot(1, N - 1, n)
    plot(times, get_spin_data(n)[1, 1:end])
    plot(times, get_spin_data(n)[2, 1:end])
    plot(times, get_spin_data(n)[3, 1:end])
    xlim(0, 1)
    ylim(-1, 1)
    xlabel("t/T")
    ylabel("n_" * string(n))
end
tight_layout()

__Considering that the final spin $n_4$ is fixed to $+1$, this also gives the correct solutions (the second one by symmetry)!__